## Indonesian Fake News Classification

### Dataset
source: https://www.kaggle.com/datasets/vijayandika/hoax-news-indonesia

### Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

from wordcloud import WordCloud, STOPWORDS
import nltk
import re

from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [1]:
import tensorflow as tf     
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    for gpu in gpus:
        print("Found a GPU with the name:", gpu)
else:
    print("Failed to detect a GPU.")

Failed to detect a GPU.


### Dataset

In [ ]:
real_news = pd.read_csv("detik.csv")
fake_news = pd.read_csv("turnbackhoax.csv")

In [ ]:
real_news['Title']

NameError: name 'real_news' is not defined

In [ ]:
real_news

In [ ]:
fake_news

In [ ]:
print("Banyak data berita benar: ", real_news.shape[0])
print("Banyak data berita palsu: ", fake_news.shape[0])

#### Histogram

In [ ]:
### Tabel Frekuensi
category_series = real_news["Category"]

# Convert to freq table
freq_table = category_series.value_counts()
display(freq_table)

### Bar Chart
# Membangun list nilai x dan y untuk digambar di bar chart
x = freq_table.index.to_list()
y = freq_table.to_list()

# mendefinisikan kanvas untuk menggambar diagram
fig = plt.figure(figsize =(8, 5)) 

# perintah untuk menggambar diagram di kanvas
plt.bar(x, y, color='grey')

# perintah untuk memberi label sumbu x dan y
plt.xlabel("Category")
plt.ylabel("Total")

# perintah untuk memberi judul diagram
plt.title("Jumlah data masing-masing category berita")

# perintah untuk menampilkan gambar di kanvas
plt.show()

#### Wordcloud Berita Benar

### Data Preprocessing

#### Penggabungan tabel

In [ ]:
real_news['Is_Fake'] = 0
fake_news['Is_Fake'] = 1
df_concatenated = pd.concat([real_news, fake_news]).reset_index(drop=True)
df_concatenated

#### Menghapus kolom yang tidak berguna
Kolom date dan link tidak akan kami pakai karena tidak memberikan pengaruh terhadap pemodelan NLP. Waktu kapan berita diunggah tidak menentukan bahwa berita tersebut hoax atau tidak.

In [ ]:
df_concatenated.drop(columns=['Date', 'Link'], inplace=True)

#### Membuat kolom tambahan untuk menggabung judul dan isi berita

In [ ]:
df_concatenated['Content'] = df_concatenated['Title'] + " " + df_concatenated['Text']
df_concatenated.head()

#### Case Folding

In [ ]:
df_concatenated['Content'] =  df_concatenated['Content'].apply(lambda x: x.lower())
df_concatenated

#### Erasing Punctuation
Pada bagian ini kita mengambil data title dan juga menghapus setiap karakter "punctuation" dan juga "bracket". Setelah kita sudah memfilter setiap karakter selain alphabet, kita akan memasukannya kedalam satu kolom yang bernama 'Content' supaya data tersebut bisa lebih mudah kita proses.

In [ ]:
### Erasing Punctuation
df_concatenated['Content'] = df_concatenated['Content'].str.replace(r'[^a-zA-Z\s]', ' ', regex=True)
df_concatenated


#### Menghilangkan Stop Words bahasa Indonesia

In [ ]:
stop_words = stopwords.words('indonesian')
stop_words.extend(['dari', 'subjek', 're', 'edu', 'penggunaan', 'salah', 'benar'])


def preprocess(text):
  result = []
  for token in gensim.utils.simple_preprocess(text):
    if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3 and token not in stop_words:
      result.append(token)
  return result


df_concatenated['Stopword_Filtered'] = df_concatenated['Content'].apply(preprocess)

print("Stopword_Filtered Version:", '\n', df_concatenated['Stopword_Filtered'][0])



#### Stemming
Stemming merupakan proses untuk menghilangkan imbuhan pada kata seperti 

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(token_list):
    return [stemmer.stem(token) for token in token_list]


df_concatenated['Stemmed'] = df_concatenated['Stopword_Filtered'].apply(stemming)

### EDA

### Modelling